In [1]:
raise ValueError('custom exception')


ValueError: custom exception

In [4]:
try:
    raise ValueError('custom message','secondary message')
except ValueError as ex:
    print(repr(ex))

ValueError('custom message', 'secondary message')


In [8]:
def func_1():
    raise TypeError('bad value')

In [9]:
try:
    func_1()
except ValueError as ex:
    print('handling a value error',repr(ex))
except IndexError as ex:
    print('handling an index error',repr(ex))
   

TypeError: bad value

In [12]:
try:
    raise TypeError('error')
except ValueError as ex:
    print('handling value exception',repr(ex))
except Exception as ex:
    print('handling exception',repr(ex))
   

handling exception TypeError('error',)


In [13]:
try:
    raise ValueError()
except ValueError:
    print('handled error')
finally:
    print('running finally')

handled error
running finally


In [14]:
try:
    pass
except ValueError:
    print('handled error')
finally:
    print('running finally')

running finally


In [15]:
try:
    raise KeyError()
except ValueError:
    print('handled error')
finally:
    print('running finally')

running finally


KeyError: 

In [16]:
try:
    a=10
except ValueError:
    print('value error')
else:
    print('no exception')

no exception


In [19]:
try:
    raise ValueError()
except ValueError:
    print('exception...')
    raise
else:
    print('no exception')

exception...


ValueError: 

In [20]:
try:
    pass
except ValueError:
    print('value error...')
else:
    print('no exception...')

no exception...


In [21]:
import json
json_data = """{
    "Sam" : {"age":28},
    "Dean" : {"age":35,"city":"Austin"},
    "Cass":{"age":"unknown"}
    
}
"""

In [22]:
data = json.loads(json_data)

In [23]:
data

{'Sam': {'age': 28},
 'Dean': {'age': 35, 'city': 'Austin'},
 'Cass': {'age': 'unknown'}}

In [24]:
class Person:
    __slots__ = 'name','_age'
    
    def __init__(self,name):
        self.name = name
        self._age = None
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,value):
        if isinstance(value,int) and value>=0:
            self._age = value
        else:
            raise ValueError('Invalid age')
            
    def __repr__(self):
        return f'Person(name = {self.name},age={self.age})'

In [25]:
persons = []

In [29]:
for name,attributes in data.items():
    try:
        p = Person(name)
        
        for attrib_name,attrib_value in attributes.items():
            try:
                setattr(p,attrib_name,attrib_value)
            except AttributeError:
                print(f'ignoring attribute: {name}.{attrib_name} = {attrib_value}')
    except ValueError as ex:
        print(f'Data for Person({name}) contains an invalid attribute value:{ex}')
 
    else:
        persons.append(p)

ignoring attribute: Dean.city = Austin
Data for Person(Cass) contains an invalid attribute value:Invalid age


In [35]:
persons = []
for name,attributes in data.items():
    p = Person(name)
        
    for attrib_name,attrib_value in attributes.items():
        skip_person = False
        try:
            setattr(p,attrib_name,attrib_value)
        except AttributeError:
            print(f'ignoring attribute: {name}.{attrib_name} = {attrib_value}')
        except ValueError as ex:
            print(f'Data for Person({name}) contains an invalid attribute value:{ex}')
            skip_person = True
            break
 
    if not skip_person:
        persons.append(p)

ignoring attribute: Dean.city = Austin
Data for Person(Cass) contains an invalid attribute value:Invalid age


In [36]:
persons

[Person(name = Sam,age=28), Person(name = Dean,age=35)]

In [37]:
def convert_int(val):
    if not isinstance(val,int):
        raise TypeError()
    if val not in (0,1):
        raise ValueError('Integer values 0 or 1 only')
    return bool(val)

In [43]:
def convert_str(val):
    if not isinstance(val,str):
        raise TypeError()
    
    val = val.casefold()
    if val in{'0','f','false'}:
        return False
    elif val in {'1','t','true'}:
        return True
    else:
        raise ValueError('Admissible string values are: F,T,True,False,0,1...')
        

In [44]:
class ConversionError(Exception):
    pass

In [45]:
def make_bool(val):
    try:
        try:
            b = convert_int(val)
        except TypeError:
            try:
                b = convert_str(val)
            except TypeError:
                raise ConversionError(f'the type is inadmissable..')
    except ValueError as ex:
        raise ConversionError(f'the value {val} cannot be converted to bool:{ex}')
    else:
        return b

In [46]:
values = [True,0,'T','false',10,'ABC',1.0]

for value in values:
    try:
        result = make_bool(value)
    except ConversionError as ex:
        result = str(ex)
    print(value,result)

True True
0 False
T True
false False
10 the value 10 cannot be converted to bool:Integer values 0 or 1 only
ABC the value ABC cannot be converted to bool:Admissible string values are: F,T,True,False,0,1...
1.0 the type is inadmissable..


In [47]:
def make_bool(val):
    if isinstance(val,int):
        if val in {0,1}:
            return bool(val)
        else: 
            raise ConversionError("Invalid integer value")
    if isinstance(val,str):
        if val.casefold() in {'1','true','t'}:
            return True
        if val.casefold() in {'0','false','f'}:
            return False
        raise ConversionError('Invalid string value')
    raise ConversionError('Invalid type')

In [48]:
values = [True,0,'T','false',10,'ABC',1.0]

for value in values:
    try:
        result = make_bool(value)
    except ConversionError as ex:
        result = str(ex)
    print(value,result)

True True
0 False
T True
false False
10 Invalid integer value
ABC Invalid string value
1.0 Invalid type


In [49]:
def get_item_forgive_me(seq,idx,default = None):
    try:
        return seq[idx]
    except (IndexError,TypeError,KeyError):
        return default


In [55]:
def get_item_ask_perm(seq,idx,default = None):
    if hasattr(seq,'__getitem__'):
        if idx< len(seq):
            return seq[idx]
    return default
    

In [56]:
get_item_forgive_me([1,2,3],10,'Nope')

'Nope'

In [57]:
get_item_ask_perm([1,2,3],10,'Nope')

'Nope'

In [58]:
get_item_forgive_me({'a':100},'a')

100

In [59]:
get_item_ask_perm({'a':100},'a')

TypeError: '<' not supported between instances of 'str' and 'int'

In [60]:
def get_item_ask_perm(seq,idx,default = None):
    if hasattr(seq,'__getitem__'):
        if isinstance(seq,dict):
            return seq.get(idx,default)
        elif isinstance(idx,int):
            if idx < len(seq):
                return seq[idx]
    return default

In [61]:
get_item_ask_perm({'a':100},'a')

100

In [62]:
get_item_ask_perm([1,2,3],0)

1

In [63]:
class ConstantSequence:
    def __init__(self,val):
        self.val = val
    
    def __getitem__(self,idx):
        return self.val

In [64]:
seq = ConstantSequence(10)

In [65]:
seq[0],seq[1]

(10, 10)

In [66]:
get_item_forgive_me(seq,10,'Nope')

10

In [67]:
get_item_ask_perm(seq,10,'Nope')

TypeError: object of type 'ConstantSequence' has no len()

In [68]:
def get_item_forgive_me(seq,idx,default = None):
    try:
        return seq[idx]
    except (IndexError,TypeError,KeyError):
        return default


In [70]:
get_item_forgive_me([1,2,3,4],slice(1,3))

[2, 3]